In [1]:
# :(

from pathlib import Path
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import pylab

In [2]:
data = []
base_dir = 'corpus_preprocessed'

for target in ['aimait', 'terribles']:
    target_label = 1 if target == 'aimait' else 0
    
    target_dir = os.path.join(base_dir, target)
    
    for author in os.listdir(target_dir):
        author_dir = os.path.join(target_dir, author)
        
        if os.path.isdir(author_dir):
            for filename in os.listdir(author_dir):
                if filename.endswith('.txt'):
                    file_path = os.path.join(author_dir, filename)
                    
                    with open(file_path, 'r', encoding='utf-8') as file:
                        text = file.read()
                    ctr[target_label] += Counter(text.split())
                    data.append({
                        'target': target_label,
                        'author': author,
                        'title': filename[:-4],
                        'text': text
                    })

df = pd.DataFrame(data)

In [3]:
display(df)

,target,author,title,text
0,1,prudentius,prud.psycho,praefatio senex fidelis primus credo uia abram...
1,1,sidonius,sidonius3,epistula sidonius auitus salus multus uinculum...
2,1,sidonius,sidonius2,epistula sidonius ecdicio salus duo nunc parit...
3,1,sidonius,sidonius1,epistula sidonius constantio salus praecipio d...
4,1,sidonius,sidonius5,epistula sidonius petronius salus audio lectit...
...,...,...,...,...
893,0,tacitus,tac.ann15,interea rex parthi uologaeses cognosco corbulo...
894,0,tacitus,tac.ann11,ualerium asiaticum bis consul quondam adulter ...
895,0,tacitus,tac.ann12,caedes messalinae convulsus princeps domus ori...
896,0,tacitus,tac.ann13,primus nouus principatus mors iunii silanus pr...


In [29]:
from sklearn.metrics import accuracy_score
import statistics

def compute_accuracy(tfidf, model, test_df):
    authors = test_df['author'].unique()
    accs = []
    for author in authors:
        df_temp = test_df[test_df['author'] == author]
        X_test = tfidf.transform(df_temp['text']).toarray()
        y_test = np.asarray(df_temp['target'])
        y_pre = model.predict(X_test)
        accs.append(accuracy_score(y_test,y_pre))

    return statistics.fmean(accs)

In [43]:
vocab_dim = [500, 1000, 5000, 10000, 20000, 30000, 50000, 100000]
ngram = [1, 2, 3, 4]
models = [] # logreg, bernoulli NB, SVM, random forest, stacking (from process), extra trees, KNN Bonus: neural networkss (LSTM) 

Принцип оценки качества моделей: выбираем случайно несколько авторов. Для каждого автора тренируем модель на всех остальных авторах, применяем модель на текстах выбранного автора, считаем среднее арифметическое по accuracy. итоговая accuracy это среднее по всем выбранным авторам.

Для чистоты эксперимента выберем авторов для валидации заранее, одних и тех же для всех моделей. 

In [4]:
authors_class_0 = df[df['target'] == 0]['author'].unique()
authors_class_1 = df[df['target'] == 1]['author'].unique()

print(f"0 - target: {len(authors_class_0)}")
print(f"1 - target: {len(authors_class_1)}")

0 - target: 34
1 - target: 32


Выберем по 6 авторов для валидации

In [14]:
import random

selected_authors_0 = random.sample(list(authors_class_0), 6)
selected_authors_1 = random.sample(list(authors_class_1), 6)

print("0 - target:", selected_authors_0)
print("1 - target:", selected_authors_1)

0 - target: ['tacitus', 'ambrosius', 'horace', 'cyprianus', 'eginhardus', 'martial']
1 - target: ['sedulius', 'petronius', 'eusebius', 'boethius', 'bonifatius', 'vitae']


In [15]:
print(f"number of 0 - target val texts: {df[df['author'].isin(selected_authors_0)].shape[0]}")
print(f"number of 1 - target val texts: {df[df['author'].isin(selected_authors_1)].shape[0]}")

number of 0 - target val texts: 109
number of 1 - target val texts: 31


In [23]:
test_df = df[df['author'].isin(selected_authors_0 + selected_authors_1)]
display(test_df)

,target,author,title,text
22,1,vitae,vita_columbani,dominus eximius sacer culmen regimen decoratis...
25,1,bonifatius,bonifatius_enigmata,enigmata p aureus decem transmitto pomum soror...
69,1,sedulius,sedulius4,sedulius carmen paschal sedulius carmen pascha...
70,1,sedulius,sedulius5,sedulius carmen paschal sedulius carmen pascha...
71,1,sedulius,sedulius1,sedulius carmen paschal sedulius carmen pascha...
...,...,...,...,...
892,0,tacitus,tac.dialogus,saepe requiro justus fabi prior saeculum tot e...
893,0,tacitus,tac.ann15,interea rex parthi uologaeses cognosco corbulo...
894,0,tacitus,tac.ann11,ualerium asiaticum bis consul quondam adulter ...
895,0,tacitus,tac.ann12,caedes messalinae convulsus princeps domus ori...


In [24]:
train_df = df[~df['author'].isin(selected_authors_0 + selected_authors_1)]
display(train_df)

,target,author,title,text
0,1,prudentius,prud.psycho,praefatio senex fidelis primus credo uia abram...
1,1,sidonius,sidonius3,epistula sidonius auitus salus multus uinculum...
2,1,sidonius,sidonius2,epistula sidonius ecdicio salus duo nunc parit...
3,1,sidonius,sidonius1,epistula sidonius constantio salus praecipio d...
4,1,sidonius,sidonius5,epistula sidonius petronius salus audio lectit...
...,...,...,...,...
827,0,cicero,fam3,scr roma exeo cicero adpio imp senatus res pub...
828,0,cicero,domo,multus divinitus pontifex magnus noster inueni...
829,0,cicero,nd2,cotta dico uelleius inquam incautus academicus...
830,0,cicero,fam14,scr dyrrhachii kal decembres tullius terentiae...


In [25]:
train_df.target.value_counts()

target
0    659
1     99
Name: count, dtype: int64

In [28]:
df_balanced = pd.concat([train_df[train_df.target == 0].sample(99), train_df[train_df.target == 1]], ignore_index = True)
df_balanced.target.value_counts()

target
0    99
1    99
Name: count, dtype: int64

In [21]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

**Оценка моделей**

Начнем по порядку оценивать качество моделей, варьируя гиперпараметры

In [22]:
from sklearn.linear_model import LogisticRegression

In [44]:
model = LogisticRegression()
logreg_list = []

for vocab in vocab_dim:
    for n in ngram:
        
        tfidf = TfidfVectorizer(max_features=vocab, ngram_range=(1, n))
        
        X = tfidf.fit_transform(df_balanced['text']).toarray()
        y = np.asarray(df_balanced['target'])
        
        model.fit(X, y)
        accuracy = compute_accuracy(tfidf, model, test_df)
        print(f"Vocabulary size: {vocab}, max ngram: {n}, accuracy: {accuracy}")
        logreg_list.append((vocab, n, accuracy))

Vocabulary size: 500, max ngram: 1, accuracy: 0.4863473672297202
Vocabulary size: 500, max ngram: 2, accuracy: 0.48541366041366046
Vocabulary size: 500, max ngram: 3, accuracy: 0.48541366041366046
Vocabulary size: 500, max ngram: 4, accuracy: 0.48541366041366046
Vocabulary size: 1000, max ngram: 1, accuracy: 0.4805272614096143
Vocabulary size: 1000, max ngram: 2, accuracy: 0.4805272614096143
Vocabulary size: 1000, max ngram: 3, accuracy: 0.4805272614096143
Vocabulary size: 1000, max ngram: 4, accuracy: 0.4805272614096143
Vocabulary size: 5000, max ngram: 1, accuracy: 0.5855236539060068
Vocabulary size: 5000, max ngram: 2, accuracy: 0.5855236539060068
Vocabulary size: 5000, max ngram: 3, accuracy: 0.5855236539060068
Vocabulary size: 5000, max ngram: 4, accuracy: 0.5855236539060068
Vocabulary size: 10000, max ngram: 1, accuracy: 0.5894919078742608
Vocabulary size: 10000, max ngram: 2, accuracy: 0.5855236539060068
Vocabulary size: 10000, max ngram: 3, accuracy: 0.5855236539060068
Vocabula

Наилучшая точность 0.589 получена при размере словаря 10000 и 30000 и размере n-грамм 1. Тем не менее, качеcтво результатов нельзя считать удовлетворительным.

In [45]:
from sklearn import svm

model = svm.SVC()
svm_list = []

for vocab in vocab_dim:
    for n in ngram:
        
        tfidf = TfidfVectorizer(max_features=vocab, ngram_range=(1, n))
        
        X = tfidf.fit_transform(df_balanced['text']).toarray()
        y = np.asarray(df_balanced['target'])
        
        model.fit(X, y)
        accuracy = compute_accuracy(tfidf, model, test_df)
        print(f"Vocabulary size: {vocab}, max ngram: {n}, accuracy: {accuracy}")
        svm_list.append((vocab, n, accuracy))

Vocabulary size: 500, max ngram: 1, accuracy: 0.4942177376000905
Vocabulary size: 500, max ngram: 2, accuracy: 0.4893157768157768
Vocabulary size: 500, max ngram: 3, accuracy: 0.4893157768157768
Vocabulary size: 500, max ngram: 4, accuracy: 0.4893157768157768
Vocabulary size: 1000, max ngram: 1, accuracy: 0.500699219081572
Vocabulary size: 1000, max ngram: 2, accuracy: 0.500699219081572
Vocabulary size: 1000, max ngram: 3, accuracy: 0.500699219081572
Vocabulary size: 1000, max ngram: 4, accuracy: 0.500699219081572
Vocabulary size: 5000, max ngram: 1, accuracy: 0.5003384687208217
Vocabulary size: 5000, max ngram: 2, accuracy: 0.5003384687208217
Vocabulary size: 5000, max ngram: 3, accuracy: 0.5003384687208217
Vocabulary size: 5000, max ngram: 4, accuracy: 0.5003384687208217
Vocabulary size: 10000, max ngram: 1, accuracy: 0.583671802054155
Vocabulary size: 10000, max ngram: 2, accuracy: 0.5003384687208217
Vocabulary size: 10000, max ngram: 3, accuracy: 0.5045051353874883
Vocabulary size:

И снова наилучший результат 0.584 достигается при размере словаря 10000 и размерен n-грамм 1.
На всякий случай проверим еще один алгоритм.

In [46]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
forest_list = []

for vocab in vocab_dim:
    for n in ngram:
        
        tfidf = TfidfVectorizer(max_features=vocab, ngram_range=(1, n))
        
        X = tfidf.fit_transform(df_balanced['text']).toarray()
        y = np.asarray(df_balanced['target'])
        
        model.fit(X, y)
        accuracy = compute_accuracy(tfidf, model, test_df)
        print(f"Vocabulary size: {vocab}, max ngram: {n}, accuracy: {accuracy}")
        forest_list.append((vocab, n, accuracy))

Vocabulary size: 500, max ngram: 1, accuracy: 0.4829984013807543
Vocabulary size: 500, max ngram: 2, accuracy: 0.5415595025889144
Vocabulary size: 500, max ngram: 3, accuracy: 0.48993506493506495
Vocabulary size: 500, max ngram: 4, accuracy: 0.504368616133322
Vocabulary size: 1000, max ngram: 1, accuracy: 0.5244521545992135
Vocabulary size: 1000, max ngram: 2, accuracy: 0.49867901706137
Vocabulary size: 1000, max ngram: 3, accuracy: 0.4648406332229862
Vocabulary size: 1000, max ngram: 4, accuracy: 0.5064531873355403
Vocabulary size: 5000, max ngram: 1, accuracy: 0.49162139603316074
Vocabulary size: 5000, max ngram: 2, accuracy: 0.5059559177206235
Vocabulary size: 5000, max ngram: 3, accuracy: 0.4840944458591518
Vocabulary size: 5000, max ngram: 4, accuracy: 0.5677722604193193
Vocabulary size: 10000, max ngram: 1, accuracy: 0.4761278753925813
Vocabulary size: 10000, max ngram: 2, accuracy: 0.4564411623235152
Vocabulary size: 10000, max ngram: 3, accuracy: 0.4668042469513058
Vocabulary s

Наилучшая точность 0.61 достигается при размере словаря 20000 и размере n-грамм 1.

Можно сделать такой вывод: либо разбиение на train и test было неудачным, либо модели на мешке слов не подходят для нашей задачи.